In [1]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [2]:
from cnn.cnn import load_dataset, create_model, steps
from dataset import dataset, load_files
from utilities.labels import LABELS
from dataset.data_augmentation import augmentated_filenames2

/home/congyu/congyu_program/pythons/forks/ECGNeuralNetwork


In [3]:
data_dir = "./data/mit_bih/"
save_dir = "./data/beats_img/"
_dataset_dir = './data/beats_img'
_model = './trained_models/cnn_aami_baseline_compare.h5'

In [4]:
train = None
validation = None
augmentation = False
_batch_size = 32
_size = (64, 64)
_epochs = 30

In [5]:
model = create_model(out_dim=4)

train, validation, test = dataset.load_files(_dataset_dir, 
                                             verbose=True, 
                                             keep_ratio=0.1)
    
print(len(train))
print(len(validation))
print(len(test))


AEB: 16	
APC: 2544	
FUS: 803	
JEB: 229	
JPB: 83	
LBB: 8072	
NOR: 75016	
PAB: 7024	
PVC: 7130	
RBB: 7256	
SVP: 2	
VEB: 106	
VFW: 472	
aPC: 150	
7622
1633
1634


In [6]:
if augmentation:
    train = augmentated_filenames2(train)

callbacks_list = [ModelCheckpoint(_model, monitor='val_loss', save_best_only=False),
                  EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
                                baseline=None, restore_best_weights=False),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=0, mode='auto')
# TrainValTensorBoard(write_graph=False)
]

data_gen_train = load_dataset(train, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

data_gen_valid = load_dataset(validation, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

model.fit_generator(data_gen_train,
                    steps_per_epoch=steps(train, _batch_size),
                    epochs=_epochs,
                    validation_data=data_gen_valid,
                    validation_steps=steps(validation, _batch_size),
                    callbacks=callbacks_list)


Epoch 1/30
239/238 [==============================] - 125s 525ms/step - loss: 0.4928 - accuracy: 0.9142 - val_loss: 0.7626 - val_accuracy: 0.9030
Epoch 2/30
239/238 [==============================] - 18s 76ms/step - loss: 0.2959 - accuracy: 0.9467 - val_loss: 0.3289 - val_accuracy: 0.9219
Epoch 3/30
239/238 [==============================] - 18s 76ms/step - loss: 0.2368 - accuracy: 0.9562 - val_loss: 0.3975 - val_accuracy: 0.9170
Epoch 4/30
239/238 [==============================] - 18s 77ms/step - loss: 0.2024 - accuracy: 0.9663 - val_loss: 0.2600 - val_accuracy: 0.8969
Epoch 5/30
239/238 [==============================] - 18s 77ms/step - loss: 0.1862 - accuracy: 0.9680 - val_loss: 0.2587 - val_accuracy: 0.9121
Epoch 6/30
239/238 [==============================] - 18s 77ms/step - loss: 0.1668 - accuracy: 0.9738 - val_loss: 0.4346 - val_accuracy: 0.9445
Epoch 7/30
239/238 [==============================] - 18s 77ms/step - loss: 0.1616 - accuracy: 0.9729 - val_loss: 0.4706 - val_accura

In [7]:
from keras.models import load_model

model = load_model(_model)

In [8]:
from evaluation import evaluate_cnn

In [9]:
evaluate_cnn(model, label_type="AAMI", keep_ratio=0.1)
print("")

changing 1530 images to arrays
predicting on 1530 samples
precision:  0.953116410942864
recall:  0.9490196078431372
f1score:  0.9485758255174623
marco f1 score 0.6737648530733986


ValueError: Number of classes, 4, does not match size of target_names, 3. Try specifying the labels parameter